# Conversational BOT with entity extraction

Goal is to create a **conversational chatbot** capable of asking questions in order to fulfil user's query and also parse all the entities required to fulfils user's request.

**Example Scenario:**
```text
USER: I want to book airline ticket
AI AGENT: Sure, Can you please provide me with the source and destination
USER: Yes, Please book ticket from A to B
AI AGENT: Sure, I have booked the ticket for you. Here's you airline ticket from A to B: <link-to-ticket>
```

# Install required libraries

In [ ]:
!pip install langchain

# Defining ENV variable for Open-AI API key

In [2]:
import os
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY_HERE" # TODO

# Defining LLM:

In [3]:
from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003") # gpt-3.5-turbo, text-davinci-003 works great

# Build a LangChain for conversational bot which is capable of extracting entities

In [26]:
from langchain.llms import OpenAI
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

# output would be expected in this format, later used in PromptTemplate
output_json_schema = """{
    "source_location": string // This is source location which is to extract from the below conversation
    "destination_location": string // This is destination location which is to extract from the below conversation
    "date": string // This is travel date location which is to extract from the below conversation
}"""

# Define the template
template = """
You are ticket booking agent, in order to book a ticket you need following information,
- source location
- destination location
- date

When user asks for booking a ticket. You should ask for above information if not already provided.
Only after you get all the above information, please extract the following entities and convert it in a valid JSON key-value pair and response must be only the json:
{output_json_schema}
                
Current conversation:\n{history}\nHuman: {input}\nAI:"""

# Defining memory
memory = ConversationBufferMemory()

# Defining conversation chain object
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=memory,
    prompt=PromptTemplate(
        input_variables=['history', 'input'],
        output_parser=None, 
        partial_variables={"output_json_schema": output_json_schema},
        template=template,
        template_format='f-string',
        validate_template=True)
)

# Conversation begins here:

In [27]:
response = conversation.predict(input="Hi there!, I want to book ticket")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:

You are ticket booking agent, in order to book a ticket you need following information,
- source location
- destination location
- date

When user asks for booking a ticket. You should ask for above information if not already provided.
Only after you get all the above information, please extract the following entities and convert it in a valid JSON key-value pair and response must be only the json:
{
    "source_location": string // This is source location which is to extract from the below conversation
    "destination_location": string // This is destination location which is to extract from the below conversation
    "date": string // This is travel date location which is to extract from the below conversation
}
                
Current conversation:

Human: Hi there!, I want to book ticket
AI:

> Finished chain.
 Hi there! Where do you want to travel from?


In [28]:
response = conversation.predict(input="I want to book ticket from Mumbai to Delhi")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:

You are ticket booking agent, in order to book a ticket you need following information,
- source location
- destination location
- date

When user asks for booking a ticket. You should ask for above information if not already provided.
Only after you get all the above information, please extract the following entities and convert it in a valid JSON key-value pair and response must be only the json:
{
    "source_location": string // This is source location which is to extract from the below conversation
    "destination_location": string // This is destination location which is to extract from the below conversation
    "date": string // This is travel date location which is to extract from the below conversation
}
                
Current conversation:
Human: Hi there!, I want to book ticket
AI:  Hi there! Where do you want to travel from?
Human: I want to book ticket from Mumbai to Delhi
AI:

> Finished chain.
 Sur

In [29]:
response = conversation.predict(input="I want to travel on 18th April, 2023")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:

You are ticket booking agent, in order to book a ticket you need following information,
- source location
- destination location
- date

When user asks for booking a ticket. You should ask for above information if not already provided.
Only after you get all the above information, please extract the following entities and convert it in a valid JSON key-value pair and response must be only the json:
{
    "source_location": string // This is source location which is to extract from the below conversation
    "destination_location": string // This is destination location which is to extract from the below conversation
    "date": string // This is travel date location which is to extract from the below conversation
}
                
Current conversation:
Human: Hi there!, I want to book ticket
AI:  Hi there! Where do you want to travel from?
Human: I want to book ticket from Mumbai to Delhi
AI:  Sure, when do you want

In [31]:
import json

# Parse output_dict from final json response obtained in conversation, which can be used as an input to an API
output_dict = json.loads(response)
output_dict

{'source_location': 'Mumbai',
 'destination_location': 'Delhi',
 'date': '18th April, 2023'}